# Video Classification

In [ ]:
import os
import cv2
import numpy as np
from imutils import paths

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Model
from pickle import dump
from keras.optimizers import SGD

In [ ]:
datapath= r"E:\RESEARCH\Datasets\VC\sports\data"
outputmodel = r"E:\RESEARCH\Datasets\VC\sports\classificationModel"
outputlabelbinarizer = "E:\RESEARCH\Datasets\VC\sports\classificationBinarizer"
epochs=25

In [ ]:
sports_labels = set(['boxing','swimming','table_tennis'])
print("Images are being loaded...")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in sports_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #if image is not in RGB form, change it to RGB format
    image = cv2.resize(image, (244, 244)) #resizing the image. Here we trying resnet50 so 244x244
    data.append(image) # appending data
    labels.append(label)

In [ ]:
data = np.array(data)
labels = np.array(labels) # hot encodded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size = 0.25, stratify=labels, random_state=42)

In [ ]:
#Image data augmentation
trainingAugmentation = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainingAugmentation.mean = mean
validationAugmentation.mean = mean

In [ ]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(244,244,3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

In [ ]:
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epochs)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

In [ ]:
History = model.fit_generator(
    trainingAugmentation.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch = len(X_train) // 32,
    validation_data = validationAugmentation.flow(X_test, Y_test),
    validation_steps = len(X_test) // 32,
    epochs=epochs
)

In [ ]:
# model.save(outputmodel)
# lbinarizer = open(r"E:\RESEARCH\Datasets\sports\classificationBinarizer.pkl", "wb")
# lbinarizer = write(pickle.dumps(lb))
# lbinarizer.close()

In [ ]:
model.save(outputmodel)
dump(outputmodel,open(r"E:\RESEARCH\Datasets\sports\classificationBinarizer.pkl", "wb"))

https://www.youtube.com/watch?v=BllyHlq5SsQ&list=PLxefhmF0pcPl_v-lLsqF3drP6NOoSYJR0

## Now, prediction

In [ ]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

In [ ]:
model = load_model(r"E:\RESEARCH\Datasets\sports\classificationModel")
lb = pickle.loads(open(r"E:\RESEARCH\Datasets\sports\classificationBinarizer.pkl","rb").read())
outputvideo = r"E:\RESEARCH\Datasets\sports\classificationModel\demo_output.avi"
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)

In [ ]:
lb.classes_[i]

In [ ]:
capture_video = cv2.VideoCapture(r"E:\RESEARCH\Datasets\sports\classificationModel\demo.mp4")
writer = None
(Width, Height) = (None, None)

while True:
    (taken, frame) = capture_video.read()
    if not taken:
        break
    if Width is None or Height is None:
        (Width, Height) = frame.shape[:2]
    
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (244, 244)).astype("float32")
    frame -= mean
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    text = "The scene is about playing {}".format(label)
    cv2.putText(output, text, (45,60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 0, 0), 5)
    
    if writer is None:
        fourcc = cv3.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("outputvideo", fourcc, 30, (Width, Height), True)
    writer.write(output)
    cv2.imshow("In progress",output)
    key = cv2.waitKet(1) & 0xFF
    
    if key == ord("q"):
        break

print("Finalizing ...")
writer.release()
capture_video.release()

In [ ]:
import os
import cv2
import imageio
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras